# 2.0 - Data Prep for Classification Model

This notebook downloads the **Fashion-MNIST** dataset and reorganizes it from its 10 default classes into our project's 8 classes.

**Our Target Classes:**
* `jacket`
* `shirt`
* `pants`
* `shorts`
* `skirt`
* `dress`
* `shoe`
* `slipper`

In [2]:
import os
import pathlib
import sys

# Get project root
CWD = pathlib.Path.cwd().resolve()
PROJECT_ROOT = CWD.parent if CWD.name == "notebooks" else CWD

DATA_DIR = PROJECT_ROOT / "data"

# Final organized dataset directory (for YOLO training)
DATASET_DIR = DATA_DIR / "dataset-fashion-modisch"
TRAIN_DIR = DATASET_DIR / "train"
VAL_DIR = DATASET_DIR / "val"

# Source Kaggle dataset paths
CLOTHES_DATASET_SLUG = "ryanbadai/clothes-dataset"
SHOE_DATASET_SLUG = "noobyogi0100/shoe-dataset"

CLOTHES_ZIP = DATA_DIR / "clothes-dataset.zip"
SHOES_ZIP = DATA_DIR / "shoe-dataset.zip"

CLOTHES_SOURCE_DIR = DATA_DIR / "clothes-dataset-unzipped"
SHOES_SOURCE_DIR = DATA_DIR / "shoe-dataset-unzipped"

# Add src to path
SRC_DIR = PROJECT_ROOT / "src"
if str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))

print(f"Project root: {PROJECT_ROOT}")
print(f"Data dir: {DATA_DIR}")
print(f"Final Dataset dir: {DATASET_DIR}")
print(f"Kaggle Clothes source dir: {CLOTHES_SOURCE_DIR}")
print(f"Kaggle Shoes source dir: {SHOES_SOURCE_DIR}")

Project root: /Users/macm4/repositories/Machine Learning Model/modisch-model-cls
Data dir: /Users/macm4/repositories/Machine Learning Model/modisch-model-cls/data
Final Dataset dir: /Users/macm4/repositories/Machine Learning Model/modisch-model-cls/data/dataset-fashion-modisch
Kaggle Clothes source dir: /Users/macm4/repositories/Machine Learning Model/modisch-model-cls/data/clothes-dataset-unzipped
Kaggle Shoes source dir: /Users/macm4/repositories/Machine Learning Model/modisch-model-cls/data/shoe-dataset-unzipped


## Step 1: Download the Data

We'll use the Ultralytics downloader to fetch the dataset.

In [16]:
import kaggle
import zipfile
import os
from pathlib import Path

# --- Authenticate and Download ---
print("Authenticating")
try:
    kaggle.api.authenticate()
    print("Authentication successful.")

    # --- Download Clothes Dataset ---
    if not CLOTHES_ZIP.exists():
        kaggle.api.dataset_download_files(
            CLOTHES_DATASET_SLUG,
            path=DATA_DIR,
            unzip=False  # Download as zip
        )
        # The Kaggle API often names the file after the dataset slug
        # We must rename it to match our CLOTHES_ZIP path
        downloaded_zip = DATA_DIR / f"{CLOTHES_DATASET_SLUG.split('/')[-1]}.zip"
        if downloaded_zip.exists() and not CLOTHES_ZIP.exists():
             downloaded_zip.rename(CLOTHES_ZIP)
        print(f"Download complete: {CLOTHES_ZIP}")
    else:
        print(f"{CLOTHES_ZIP.name} already downloaded.")

    # --- Download Shoe Dataset ---
    if not SHOES_ZIP.exists():
        print(f"Downloading {SHOE_DATASET_SLUG}...")
        kaggle.api.dataset_download_files(
            SHOE_DATASET_SLUG,
            path=DATA_DIR,
            unzip=False  # Download as zip
        )
        # Rename this zip file as well
        downloaded_zip = DATA_DIR / f"{SHOE_DATASET_SLUG.split('/')[-1]}.zip"
        if downloaded_zip.exists() and not SHOES_ZIP.exists():
             downloaded_zip.rename(SHOES_ZIP)
        print(f"Download complete: {SHOES_ZIP}")
    else:
        print(f"{SHOES_ZIP.name} already downloaded.")

except Exception as e:
    print(f"Error: {e}")

# Unzip Clothes Dataset
CLOTHES_SOURCE_DIR.mkdir(parents=True, exist_ok=True)
if not any(CLOTHES_SOURCE_DIR.iterdir()) and CLOTHES_ZIP.exists():
    print(f"Unzipping {CLOTHES_ZIP.name}...")
    with zipfile.ZipFile(CLOTHES_ZIP, 'r') as zf:
        zf.extractall(CLOTHES_SOURCE_DIR)
    print(f"Unzip complete to: {CLOTHES_SOURCE_DIR}")
elif any(CLOTHES_SOURCE_DIR.iterdir()):
    print(f"Clothes dataset already unzipped at: {CLOTHES_SOURCE_DIR}")
else:
    print(f"Could not find {CLOTHES_ZIP.name} to unzip.")

# Unzip Shoe Dataset
SHOES_SOURCE_DIR.mkdir(parents=True, exist_ok=True)
if not any(SHOES_SOURCE_DIR.iterdir()) and SHOES_ZIP.exists():
    print(f"Unzipping {SHOES_ZIP.name}...")
    with zipfile.ZipFile(SHOES_ZIP, 'r') as zf:
        zf.extractall(SHOES_SOURCE_DIR)
    print(f"Unzip complete to: {SHOES_SOURCE_DIR}")
elif any(SHOES_SOURCE_DIR.iterdir()):
    print(f"Shoe dataset already unzipped at: {SHOES_SOURCE_DIR}")
else:
    print(f"Could not find {SHOES_ZIP.name} to unzip.")

Authenticating
Authentication successful.
Dataset URL: https://www.kaggle.com/datasets/ryanbadai/clothes-dataset
Download complete: /Users/macm4/repositories/Machine Learning Model/modisch-model-cls/data/clothes-dataset.zip
Dataset URL: https://www.kaggle.com/datasets/noobyogi0100/shoe-dataset
Download complete: /Users/macm4/repositories/Machine Learning Model/modisch-model-cls/data/clothes-dataset.zip
Dataset URL: https://www.kaggle.com/datasets/noobyogi0100/shoe-dataset
Download complete: /Users/macm4/repositories/Machine Learning Model/modisch-model-cls/data/shoe-dataset.zip
Unzipping clothes-dataset.zip...
Download complete: /Users/macm4/repositories/Machine Learning Model/modisch-model-cls/data/shoe-dataset.zip
Unzipping clothes-dataset.zip...
Unzip complete to: /Users/macm4/repositories/Machine Learning Model/modisch-model-cls/data/clothes-dataset-unzipped
Unzipping shoe-dataset.zip...
Unzip complete to: /Users/macm4/repositories/Machine Learning Model/modisch-model-cls/data/clot

## Step 3: Create Your New Class Folders

This creates the `train` and `val` directories, each containing your 8 target class folders.

In [3]:
# target class
CLASSES_TO_CREATE = [
    'boot', 'dress', 'pants', 'shirt', 'sneaker', 'flip-flop', 'loafer',
    'short', 'skirt', 'slipper', 't-shirt', 'blazer', 'hoodie', 'jacket', 'sweater', 'polo'
]

print(f"Creating new directory structure for {len(CLASSES_TO_CREATE)} classes...")
for split in [TRAIN_DIR, VAL_DIR]:
    for cls_name in CLASSES_TO_CREATE:
        os.makedirs(split / cls_name, exist_ok=True)

print("New structure created.")

Creating new directory structure for 16 classes...
New structure created.


## Step 4: "Re-label" by Moving Files

This is the key step. We use terminal commands (`mv`) to move all images from the Fashion-MNIST folders (e.g., `T-shirt/top`) into your new folders (e.g., `shirt`).

This works because we are in the `notebooks/` directory, so `../fashion-mnist` points to the unzipped folder.

In [4]:
import shutil
import random
import glob
from pathlib import Path

print("Mapping new Kaggle datasets and splitting into train/val...")

# --- Mappings from Kaggle source folders to your 16 target class names ---

# Target classes (as defined in the cell above)
TARGET_CLASSES = set(CLASSES_TO_CREATE)

# 1. ryanbadai/clothes-dataset
CLOTHES_BASE_DIR = CLOTHES_SOURCE_DIR

CLOTHES_MAP = {
    "Blazer": "blazer",
    "Celana_Panjang": "pants",
    "Celana_Pendek": "short",
    "Gaun": "dress",
    "Hoodie": "hoodie",
    "Jaket": "jacket",
    "Jaket_Denim": "jacket",       # Grouped into 'jacket'
    "Jaket_Olahraga": "jacket",    # Grouped into 'jacket'
    "Jeans": "pants",          # Grouped into 'pants'
    "Kaos": "t-shirt",
    "Kemeja": "shirt",
    "Mantel": "jacket",        # Grouped into 'jacket'
    "Polo": "polo",
    "Rok": "skirt",
    "Sweter": "sweater",
}

# 2. noobyogi0100/shoe-dataset
SHOES_BASE_DIR = SHOES_SOURCE_DIR

SHOES_MAP = {
    "boots": "boot",
    "sneakers": "sneaker",
    "flip flops": "flip-flop",
    "loafers": "loafer",
    "sandals": "slipper",      # Mapped to 'slipper'
    "soccer shoes": "sneaker",     # Grouped into 'sneaker'
}
# --- End Mappings ---


def split_and_copy_files(source_class_dir, target_class_name, train_dir, val_dir, split_ratio=0.8):
    """Copies files from source dir to train/val dirs with a split."""
    
    if target_class_name not in TARGET_CLASSES:
        print(f"  [Skipping] Source '{source_class_dir.name}' maps to '{target_class_name}', which is not in TARGET_CLASSES.")
        return 0, 0

    # Find all images (jpg, png, jpeg)
    images = []
    for ext in ("*.jpg", "*.jpeg", "*.png"):
        images.extend(glob.glob(str(source_class_dir / ext)))
    
    if not images:
        print(f"  [Warning] No images found in {source_class_dir}")
        return 0, 0

    random.seed(42) # for reproducible splits
    random.shuffle(images)
    split_point = int(len(images) * split_ratio)
    train_files = images[:split_point]
    val_files = images[split_point:]

    # Get target directories
    target_train_dir = train_dir / target_class_name
    target_val_dir = val_dir / target_class_name

    # Copy files
    for f in train_files:
        shutil.copy(f, target_train_dir / Path(f).name)
    for f in val_files:
        shutil.copy(f, target_val_dir / Path(f).name)
        
    return len(train_files), len(val_files)

total_train = 0
total_val = 0

# --- Process Clothes Dataset ---
print(f"\nProcessing Clothes Dataset from: {CLOTHES_BASE_DIR}")
for source_name, target_name in CLOTHES_MAP.items():
    source_dir = CLOTHES_BASE_DIR / source_name
    # Check for nested "Clothes_Dataset" folder
    if not source_dir.exists():
        nested_dir = CLOTHES_BASE_DIR / "Clothes_Dataset" / source_name
        if nested_dir.exists():
            source_dir = nested_dir
        else:
            print(f"  [Warning] Source folder not found: {source_dir.name}")
            continue
            
    print(f"Processing '{source_name}' -> '{target_name}'...")
    n_train, n_val = split_and_copy_files(source_dir, target_name, TRAIN_DIR, VAL_DIR)
    print(f"  Copied {n_train} train, {n_val} val files.")
    total_train += n_train
    total_val += n_val

# --- Process Shoe Dataset ---
print(f"\nProcessing Shoe Dataset from: {SHOES_BASE_DIR}")
for source_name, target_name in SHOES_MAP.items():
    source_dir = SHOES_BASE_DIR / source_name
    if not source_dir.exists():
        print(f"  [Warning] Source folder not found: {source_dir.name}")
        continue
        
    print(f"Processing '{source_name}' -> '{target_name}'...")
    n_train, n_val = split_and_copy_files(source_dir, target_name, TRAIN_DIR, VAL_DIR)
    print(f"  Copied {n_train} train, {n_val} val files.")
    total_train += n_train
    total_val += n_val

print("\n--- File Move Complete ---")
print(f"Total Train Images: {total_train}")
print(f"Total Val Images: {total_val}")

Mapping new Kaggle datasets and splitting into train/val...

Processing Clothes Dataset from: /Users/macm4/repositories/Machine Learning Model/modisch-model-cls/data/clothes-dataset-unzipped
  [Warning] Source folder not found: Blazer
  [Warning] Source folder not found: Celana_Panjang
  [Warning] Source folder not found: Celana_Pendek
  [Warning] Source folder not found: Gaun
  [Warning] Source folder not found: Hoodie
  [Warning] Source folder not found: Jaket
  [Warning] Source folder not found: Jaket_Denim
  [Warning] Source folder not found: Jaket_Olahraga
  [Warning] Source folder not found: Jeans
  [Warning] Source folder not found: Kaos
  [Warning] Source folder not found: Kemeja
  [Warning] Source folder not found: Mantel
  [Warning] Source folder not found: Polo
  [Warning] Source folder not found: Rok
  [Warning] Source folder not found: Sweter

Processing Shoe Dataset from: /Users/macm4/repositories/Machine Learning Model/modisch-model-cls/data/shoe-dataset-unzipped
  [Warn

## Step 5: IMPORTANT - Check for Missing Classes

The Fashion-MNIST dataset **does not contain** images for `shorts`, `skirt`, or `slipper`.

Your folders for these classes are **empty**! You will need to find images for these classes and add them to the `train/` and `val/` subfolders yourself.

In [6]:
# Step 5d: Oversample imbalanced classes in TRAIN_DIR (skirt, short/shorts)
import os
import random
import shutil
import pathlib

TRAIN_DIR = pathlib.Path(TRAIN_DIR)  # ensure Path
random.seed(42)

TARGETS = [
    'boot', 'dress', 'pants', 'shirt', 'sneaker', 'flip-flop', 'loafer',
    'short', 'skirt', 'slipper', 't-shirt', 'blazer', 'hoodie', 'sweater', 'polo'
]
ALLOWED_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".webp"}

# Optional PIL-based augmentation
try:
    from PIL import Image, ImageEnhance, ImageOps
    HAVE_PIL = True
except Exception:
    HAVE_PIL = False

def _unique_path(dirpath: pathlib.Path, stem: str, ext: str) -> pathlib.Path:
    i = 0
    while True:
        name = f"{stem}_aug{'' if i==0 else f'_{i}'}{ext}"
        out = dirpath / name
        if not out.exists():
            return out
        i += 1

def _list_images(dirpath: pathlib.Path):
    return [p for p in dirpath.glob("*") if p.is_file() and p.suffix.lower() in ALLOWED_EXTS]

def _pil_augment(img: "Image.Image") -> "Image.Image":
    # simple, fast, safe transforms
    if random.random() < 0.5:
        img = ImageOps.mirror(img)
    angle = random.uniform(-12, 12)
    img = img.rotate(angle, resample=Image.BICUBIC, expand=False, fillcolor=(255, 255, 255))
    # slight brightness/contrast jitter
    img = ImageEnhance.Brightness(img).enhance(random.uniform(0.9, 1.1))
    img = ImageEnhance.Contrast(img).enhance(random.uniform(0.9, 1.1))
    return img

def _augment_or_copy(src: pathlib.Path, dst: pathlib.Path):
    if HAVE_PIL:
        try:
            with Image.open(src) as im:
                if im.mode not in ("RGB", "L"):
                    im = im.convert("RGB")
                # Save as same ext if supported; else default to .jpg
                ext = dst.suffix.lower()
                im = _pil_augment(im)
                params = {}
                if ext in {".jpg", ".jpeg"}:
                    params = {"quality": 90, "optimize": True}
                im.save(dst, **params)
                return
        except Exception as _:
            pass
    # Fallback to raw copy
    shutil.copy2(src, dst)


In [7]:

# Count current train images per class (based on actual folders)
class_dirs = [d for d in TRAIN_DIR.iterdir() if d.is_dir()]
counts = {d.name: len(_list_images(d)) for d in class_dirs}
print("Current train counts:")
for k in sorted(counts):
    print(f" - {k:10s}: {counts[k]}")

if not counts:
    raise RuntimeError("No training classes found in TRAIN_DIR.")

target_count = max(counts.values())
targets_existing = [c for c in TARGETS if (TRAIN_DIR / c).exists()]
if not targets_existing:
    print("No target classes (skirt/short/shorts) found in TRAIN_DIR. Skipping oversampling.")
else:
    print(f"\nOversampling to target count = {target_count}")
    for cls in targets_existing:
        cls_dir = TRAIN_DIR / cls
        imgs = _list_images(cls_dir)
        n = len(imgs)
        if n == 0:
            print(f" - {cls}: no images to oversample from; add some first.")
            continue
        need = target_count - n
        if need <= 0:
            print(f" - {cls}: already >= target ({n} >= {target_count}).")
            continue

        print(f" - {cls}: adding {need} samples...")
        i = 0
        while i < need:
            src = random.choice(imgs)
            stem = src.stem
            ext = src.suffix.lower()
            if ext not in ALLOWED_EXTS:
                ext = ".jpg"
            dst = _unique_path(cls_dir, stem=stem, ext=ext)
            _augment_or_copy(src, dst)
            i += 1

# Re-count after oversampling
counts_after = {d.name: len(_list_images(d)) for d in class_dirs}
print("\nTrain counts after oversampling:")
for k in sorted(counts_after):
    print(f" - {k:10s}: {counts_after[k]}")

Current train counts:
 - blazer    : 500
 - boot      : 249
 - dress     : 500
 - flip-flop : 249
 - hoodie    : 500
 - jacket    : 1995
 - loafer    : 249
 - pants     : 1000
 - polo      : 500
 - shirt     : 500
 - short     : 500
 - skirt     : 709
 - slipper   : 249
 - sneaker   : 478
 - sweater   : 500
 - t-shirt   : 500

Oversampling to target count = 1995
 - boot: adding 1746 samples...
 - dress: adding 1495 samples...
 - pants: adding 995 samples...
 - shirt: adding 1495 samples...
 - sneaker: adding 1517 samples...
 - flip-flop: adding 1746 samples...
 - loafer: adding 1746 samples...
 - short: adding 1495 samples...
 - skirt: adding 1286 samples...
 - slipper: adding 1746 samples...
 - t-shirt: adding 1495 samples...
 - blazer: adding 1495 samples...
 - hoodie: adding 1495 samples...
 - sweater: adding 1495 samples...
 - polo: adding 1495 samples...

Train counts after oversampling:
 - blazer    : 1995
 - boot      : 1995
 - dress     : 1995
 - flip-flop : 1995
 - hoodie    :

In [8]:
# Count per-class files in train/ and val/
from collections import defaultdict

def count_files(dirpath: pathlib.Path) -> dict:
    counts = {}
    for cls in CLASSES_TO_CREATE:
        d = dirpath / cls
        n = len([f for f in d.glob("*") if f.is_file()]) if d.exists() else 0
        counts[cls] = n
    return counts

train_counts = count_files(TRAIN_DIR)
val_counts = count_files(VAL_DIR)

print("Class counts (train):")
for k, v in train_counts.items():
    print(f" - {k:8s}: {v}")

print("\nClass counts (val):")
for k, v in val_counts.items():
    print(f" - {k:8s}: {v}")

Class counts (train):
 - boot    : 1995
 - dress   : 1995
 - pants   : 1995
 - shirt   : 1995
 - sneaker : 1996
 - flip-flop: 1995
 - loafer  : 1995
 - short   : 1995
 - skirt   : 1995
 - slipper : 1995
 - t-shirt : 1995
 - blazer  : 1995
 - hoodie  : 1995
 - jacket  : 1995
 - sweater : 1995
 - polo    : 1995

Class counts (val):
 - boot    : 50
 - dress   : 0
 - pants   : 0
 - shirt   : 0
 - sneaker : 95
 - flip-flop: 50
 - loafer  : 50
 - short   : 0
 - skirt   : 0
 - slipper : 50
 - t-shirt : 0
 - blazer  : 0
 - hoodie  : 0
 - jacket  : 0
 - sweater : 0
 - polo    : 0


# TRAINING

In [8]:
# --- Dataset sanity print: list classes in train/ and val/ (no syncing/moving) ---
from pathlib import Path

train_dir = TRAIN_DIR
val_dir = VAL_DIR

train_classes = sorted([d.name for d in Path(train_dir).iterdir() if d.is_dir()])
val_classes = sorted([d.name for d in Path(val_dir).iterdir() if d.is_dir()])

print("Train classes:", train_classes)
print("Val classes:", val_classes)

Train classes: ['boot', 'dress', 'outer', 'pants', 'shirt', 'shoe', 'short', 'shorts', 'skirt', 'slipper', 't-shirt']
Val classes: ['boot', 'dress', 'outer', 'pants', 'shirt', 'shoe', 'short', 'shorts', 'skirt', 'slipper', 't-shirt']


In [9]:
random.seed(42)

ALLOWED_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".webp"}

def list_images(dirpath: Path):
    return [p for p in dirpath.glob("*") if p.is_file() and p.suffix.lower() in ALLOWED_EXTS]

train_dir = Path(TRAIN_DIR)
val_dir = Path(VAL_DIR)

train_classes = {d.name for d in train_dir.iterdir() if d.is_dir()}
val_classes = {d.name for d in val_dir.iterdir() if d.is_dir()}
all_classes = sorted(train_classes | val_classes)

# Make sure each class exists in both splits
for cls in all_classes:
    (train_dir / cls).mkdir(parents=True, exist_ok=True)
    (val_dir / cls).mkdir(parents=True, exist_ok=True)

# Ensure at least 1 image per class per split by borrowing from the other split if needed
fixed = []
for cls in all_classes:
    tr_imgs = list_images(train_dir / cls)
    va_imgs = list_images(val_dir / cls)

    if len(tr_imgs) == 0 and len(va_imgs) > 0:
        src = random.choice(va_imgs)
        dst = (train_dir / cls) / src.name
        c = 1
        while dst.exists():
            dst = dst.with_name(f"{src.stem}_{c}{src.suffix}")
            c += 1
        shutil.copy2(src, dst)
        fixed.append(f"train/{cls}")

    if len(va_imgs) == 0 and len(tr_imgs) > 0:
        src = random.choice(tr_imgs)
        dst = (val_dir / cls) / src.name
        c = 1
        while dst.exists():
            dst = dst.with_name(f"{src.stem}_{c}{src.suffix}")
            c += 1
        shutil.copy2(src, dst)
        fixed.append(f"val/{cls}")

print("Classes:", all_classes)
print("Fixed empty splits:", fixed if fixed else "none")

Classes: ['boot', 'dress', 'outer', 'pants', 'shirt', 'shoe', 'short', 'shorts', 'skirt', 'slipper', 't-shirt']
Fixed empty splits: none


In [10]:
# --- Per-epoch reporting WITHOUT touching DEFAULT_CFG (prevents YAML RepresenterError) ---
from pathlib import Path

REPORT_CSV = PROJECT_ROOT / "runs-cls" / "epoch_report.csv"

def on_fit_epoch_end(trainer):
    csv_path = Path(trainer.save_dir) / "results.csv"
    if not csv_path.exists():
        return
    last = csv_path.read_text().strip().splitlines()[-1]
    print(f"[epoch {trainer.epoch + 1}] {last}")
    if not REPORT_CSV.exists():
        header = csv_path.read_text().splitlines()[0]
        REPORT_CSV.write_text(header + "\n")
    with REPORT_CSV.open("a") as f:
        f.write(last + "\n")

def register_callbacks(model):
    # avoid duplicate registration if re-running cells
    try:
        model.remove_callback("on_fit_epoch_end")
    except Exception:
        pass
    model.add_callback("on_fit_epoch_end", on_fit_epoch_end)
    print("Per-epoch report callback registered. Writing mirror CSV to:", REPORT_CSV)

In [11]:
# --- Train YOLOv8n-cls (MPS if available) ---
from ultralytics import YOLO
from ultralytics.utils import DEFAULT_CFG
import os
import torch
from pathlib import Path

os.environ["WANDB_DISABLED"] = "true"

# Remove any default callbacks so the 'callbacks' key doesn't leak into args.yaml/cfg
try:
    if hasattr(DEFAULT_CFG, "callbacks"):
        delattr(DEFAULT_CFG, "callbacks")
except Exception:
    pass

DEVICE = "mps" if torch.backends.mps.is_available() else "cpu"
NUM_WORKERS = 4
BATCH = 64

print("Using device:", DEVICE)

model = YOLO("yolov8n-cls.pt")

# Extra safety: strip any 'callbacks' from model overrides if present
try:
    if hasattr(model, "overrides") and isinstance(model.overrides, dict):
        model.overrides.pop("callbacks", None)
except Exception:
    pass

# Register console/CSV epoch reporter on the model only (not in DEFAULT_CFG)
try:
    register_callbacks(model)
except NameError:
    REPORT_CSV = PROJECT_ROOT / "runs-cls" / "epoch_report.csv"
    def _on_fit_epoch_end(trainer):
        csv_path = Path(trainer.save_dir) / "results.csv"
        if not csv_path.exists():
            return
        last = csv_path.read_text().strip().splitlines()[-1]
        print(f"[epoch {trainer.epoch + 1}] {last}")
        if not REPORT_CSV.exists():
            header = csv_path.read_text().splitlines()[0]
            REPORT_CSV.write_text(header + "\n")
        with REPORT_CSV.open("a") as f:
            f.write(last + "\n")
    try:
        model.remove_callback("on_fit_epoch_end")
    except Exception:
        pass
    model.add_callback("on_fit_epoch_end", _on_fit_epoch_end)
    print("Per-epoch report callback registered. Writing mirror CSV to:", REPORT_CSV)

RUNS_DIR = PROJECT_ROOT / "runs-cls"
NAME = "yolov8n-cls-fashion"
EPOCHS = 30
IMGSZ = 224
SEED = 42

results = model.train(
    data=str(DATASET_DIR),
    epochs=EPOCHS,
    imgsz=IMGSZ,
    batch=BATCH,
    project=str(RUNS_DIR),
    name=NAME,
    seed=SEED,
    patience=10,         # early stopping
    verbose=True,
    device=DEVICE,
    workers=NUM_WORKERS,
    plots=False,
)

best_path = RUNS_DIR / NAME / "weights" / "best.pt"
print("Best weights:", best_path if best_path.exists() else "see run dir for weights")

Using device: mps
Per-epoch report callback registered. Writing mirror CSV to: /Users/macm4/repositories/Machine Learning Model/modisch-model-cls/runs-cls/epoch_report.csv
Per-epoch report callback registered. Writing mirror CSV to: /Users/macm4/repositories/Machine Learning Model/modisch-model-cls/runs-cls/epoch_report.csv
Ultralytics 8.3.223 🚀 Python-3.11.14 torch-2.9.0 MPS (Apple M4)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/Users/macm4/repositories/Machine Learning Model/modisch-model-cls/data/dataset-fashion-modisch, degrees=0.0, deterministic=True, device=mps, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015,

In [12]:
metrics = model.val(
    data=str(DATASET_DIR),
    imgsz=IMGSZ,
    batch=BATCH,
    plots=True,
    project=str(RUNS_DIR),
    name=NAME + "-val",
)

try:
    print(f"top1: {metrics.top1:.4f}, top5: {metrics.top5:.4f}")
except Exception:
    print("Validation metrics:", metrics)
print("Val plots saved to:", RUNS_DIR / (NAME + "-val"))

Ultralytics 8.3.223 🚀 Python-3.11.14 torch-2.9.0 CPU (Apple M4)
YOLOv8n-cls summary (fused): 30 layers, 1,448,971 parameters, 0 gradients, 3.3 GFLOPs
YOLOv8n-cls summary (fused): 30 layers, 1,448,971 parameters, 0 gradients, 3.3 GFLOPs
ERROR ❌ train: /Users/macm4/repositories/Machine Learning Model/modisch-model-cls/data/dataset-fashion-modisch/train... found 78000 images in 10 classes (requires 11 classes, not 10)
ERROR ❌ train: /Users/macm4/repositories/Machine Learning Model/modisch-model-cls/data/dataset-fashion-modisch/train... found 78000 images in 10 classes (requires 11 classes, not 10)
ERROR ❌ val: /Users/macm4/repositories/Machine Learning Model/modisch-model-cls/data/dataset-fashion-modisch/val... found 9042 images in 10 classes (requires 11 classes, not 10)
test: None...
ERROR ❌ val: /Users/macm4/repositories/Machine Learning Model/modisch-model-cls/data/dataset-fashion-modisch/val... found 9042 images in 10 classes (requires 11 classes, not 10)
test: None...
val: Fast imag

In [14]:
TEST_DIR = PROJECT_ROOT / "test"

preds = model.predict(
    source=str(TEST_DIR),
    imgsz=IMGSZ,
    save=True,
    project=str(RUNS_DIR),
    name=NAME + "-pred",
    verbose=False,
)
print("Saved predictions to:", RUNS_DIR / (NAME + "-pred"))

# Optional exports for deployment
try:
    model.export(format="onnx", opset=12)
    model.export(format="torchscript")
    print("Exported ONNX and TorchScript.")
except Exception as e:
    print("Export skipped:", e)

Results saved to /Users/macm4/repositories/Machine Learning Model/modisch-model-cls/runs-cls/yolov8n-cls-fashion-pred
Saved predictions to: /Users/macm4/repositories/Machine Learning Model/modisch-model-cls/runs-cls/yolov8n-cls-fashion-pred
Ultralytics 8.3.223 🚀 Python-3.11.14 torch-2.9.0 CPU (Apple M4)
Saved predictions to: /Users/macm4/repositories/Machine Learning Model/modisch-model-cls/runs-cls/yolov8n-cls-fashion-pred
Ultralytics 8.3.223 🚀 Python-3.11.14 torch-2.9.0 CPU (Apple M4)

PyTorch: starting from '/Users/macm4/repositories/Machine Learning Model/modisch-model-cls/runs-cls/yolov8n-cls-fashion5/weights/best.pt' with input shape (1, 3, 224, 224) BCHW and output shape(s) (1, 11) (2.8 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnxslim>=0.1.71', 'onnxruntime'] not found, attempting AutoUpdate...

PyTorch: starting from '/Users/macm4/repositories/Machine Learning Model/modisch-model-cls/runs-cls/yolov8n-cls-fashion5/weights/best.pt' with input shape (1, 3, 224